In [4]:
import os
from langchain.chat_models import init_chat_model
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
model=init_chat_model("groq:qwen/qwen3-32b")
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x1143a27b0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1143a3230>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from pydantic import BaseModel,Field

class Movie(BaseModel):
    title:str=Field(description="The title of the movie")
    year:int=Field(description="The year the movie was released")
    director:str=Field(description="The director of the movie")
    rating:float=Field(description="The movies rating out of 10")

In [6]:
model_with_structure=model.with_structured_output(Movie)
model_with_structure

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x1143a27b0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1143a3230>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Movie', 'description': '', 'parameters': {'properties': {'title': {'description': 'The title of the movie', 'type': 'string'}, 'year': {'description': 'The year the movie was released', 'type': 'integer'}, 'director': {'description': 'The director of the movie', 'type': 'string'}, 'rating': {'description': 'The movies rating out of 10', 'type': 'number'}}, 'required': ['title', 'year', 'director', 'rat

In [7]:
model.invoke("provide the details about the movie inception")

AIMessage(content='<think>\nOkay, I need to provide details about the movie "Inception." Let me start by recalling what I know about it. Directed by Christopher Nolan, right? It\'s a sci-fi action film that came out in 2010. The main character is Dom Cobb, played by Leonardo DiCaprio. The plot involves dreams and the concept of planting or extracting ideas. There\'s a term for that—maybe "inception." \n\nI remember that the story is about a thief who steals information by entering the subconscious of his targets. He\'s offered a job to erase his criminal past by performing the reverse: planting an idea in someone\'s mind. The team includes people with different roles—someone who can manipulate the dreamer\'s subconscious, a forger, another thief. There\'s a big action scene where the city folds in on itself. The music is by Hans Zimmer, and there\'s a scene where the villain is in a hotel that keeps changing. \n\nWait, there\'s something about a spinning top at the end to determine whe

In [8]:
model_with_structure.invoke("provide the details about the movie inception")

Movie(title='Inception', year=2010, director='Christopher Nolan', rating=8.8)

In [9]:
from pydantic import BaseModel,Field

class Movie(BaseModel):
    """A movie with detail."""
    title:str=Field( description="The title of the movie")
    year:int=Field(description="The year the movie was released")
    director:str=Field(description="The director of the movie")
    rating:float=Field(description="The movies rating out of 10")
 
model_with_structure = model.with_structured_output(Movie,include_raw=True)

response = model_with_structure.invoke("provide details about the movie inception")
response

{'raw': AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for details about the movie Inception. Let me check what functions I have available. There\'s a Movie function that requires title, year, director, and rating. I need to make sure I have all that information for Inception. The title is obviously "Inception". The year it was released was 2010. The director is Christopher Nolan. As for the rating, I think it\'s around 8.8 on IMDb. Let me confirm that. Yes, IMDb gives it an 8.8/10. So I should structure the tool call with those parameters. Let me double-check each required field to ensure I didn\'t miss anything. Title, year, director, and rating are all required. Alright, that\'s everything. Time to put it into the JSON format as specified.\n', 'tool_calls': [{'id': '9r0pse472', 'function': {'arguments': '{"director":"Christopher Nolan","rating":8.8,"title":"Inception","year":2010}', 'name': 'Movie'}, 'type': 'function'}]}, response_metadata={

In [10]:
from typing import Optional
from pydantic import BaseModel, Field

class Actor(BaseModel):
    name: str
    role: str

class MovieDetails(BaseModel):
    title: str
    year: int
    cast: list[Actor]
    genres: list[str]
    budget: Optional[float] = Field(
        default=None,
        description="budget in million USD"
    )


In [11]:
model_with_structure = model.with_structured_output(MovieDetails)

response = model_with_structure.invoke("provide details about the movie inception")
response

MovieDetails(title='Inception', year=2010, cast=[Actor(name='Leonardo DiCaprio', role='Dom Cobb'), Actor(name='Joseph Gordon-Levitt', role='Arthur'), Actor(name='Ellen Page', role='Ariadne'), Actor(name='Tom Hardy', role='Eames')], genres=['Action', 'Sci-Fi', 'Thriller'], budget=160.0)

In [18]:
##typed dict
from typing_extensions import TypedDict , Annotated

In [13]:
class MovieDict(TypedDict):
    """A movie with detail."""
    title: Annotated[str,"The title of the movie"]
    year: Annotated[int,"The year the movie was released"]
    director: Annotated[str,"The director of the movie"]
    rating: Annotated[float,"The movies rating out of 10"]
 

In [14]:
model_withtypedict=model.with_structured_output(MovieDict)
response=model_withtypedict.invoke("please provide the details of the movie avengers")
response

{'director': 'Joss Whedon', 'rating': 8.1, 'title': 'Avengers', 'year': 2012}

In [16]:
class Actor(TypedDict):
    name: str
    role: str

class MovieDetails(TypedDict):
    title: str
    year: int
    cast: list[Actor]
    genres: list[str]
    budget: Optional[float] = Field(
        default=None,
        description="budget in million USD"
    )
    
model_withtypedict=model.with_structured_output(MovieDetails)
response=model_withtypedict.invoke("please provide the details of the movie avengers")
response



{'budget': 220400000,
 'cast': [{'name': 'Robert Downey Jr.', 'role': 'Tony Stark / Iron Man'},
  {'name': 'Chris Evans', 'role': 'Steve Rogers / Captain America'},
  {'name': 'Mark Ruffalo', 'role': 'Bruce Banner / Hulk'},
  {'name': 'Chris Hemsworth', 'role': 'Thor'},
  {'name': 'Scarlett Johansson', 'role': 'Natasha Romanoff / Black Widow'},
  {'name': 'Jeremy Renner', 'role': 'Clint Barton / Hawkeye'}],
 'genres': ['Action', 'Science Fiction', 'Superhero'],
 'title': 'Avengers',
 'year': 2012}

In [17]:
### data classes